<a href="https://colab.research.google.com/github/Aleskies/Clasificacion-multilabel/blob/master/desafio_clasificacion_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [111]:
import pandas as pd
import numpy as np
import os


import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier


#para tratamiento de texto
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#para visualizacion
import matplotlib.pyplot as plt

# metricas clasificacion
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [4]:
os.chdir('/content/drive/MyDrive/desafio-entrevistas-trabajo/brain-food-rn-clasicador-multiclase/')

In [5]:
# 
dir_base = "/content/drive/MyDrive/desafio-entrevistas-trabajo/brain-food-rn-clasicador-multiclase/" 

df_train = pd.read_csv(dir_base +"/data/text_classification_train.csv" )
df_test = pd.read_csv(dir_base +"/data/text_classification_test.csv" )
df_emotions = pd.read_csv(dir_base +"/data/emotions.txt",  header=None)

In [6]:
df_train.shape

(43410, 3)

In [7]:
df_train.head()

,text,emotion,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


Al observar la variable objetivo `emotion`, vemos codificado las emociones con valores numéricos. 

A continuación se evidencia que los valores de las emociones no solo estan asignadas a un solo valor, además existen combinaciones de emociones en algunas de las observaciones.  

In [ ]:
df_train['emotion'].unique()

In [9]:
df_train['emotion'].describe()

count     43410
unique      711
top          27
freq      12823
Name: emotion, dtype: object

Se presentan 711 valores únicos, cantidad muy diferente a las 28 emociones que estan presentes en la lista del archivo `emotions.txt` 

In [ ]:
df_emotions

,0
0,admiration
1,amusement
2,anger
3,annoyance
4,approval
5,caring
6,confusion
7,curiosity
8,desire
9,disappointment


Debido a que una observación puede o no tener más de un sentimiento asignado, estamos en presencia de una clasificación de multiples etiquetas. 

A continuación se crea una nueva columna para asignar el sentimiento correspondiente al valor numérico asociado.

In [16]:
# Definir un diccionario que asocie cada número con su etiqueta correspondiente
etiquetas = {
    '0': 'admiration',
    '1': 'amusement',
    '2': 'anger',
    '3': 'annoyance',
    '4': 'approval',
    '5': 'caring',
    '6': 'confusion',
    '7': 'curiosity',
    '8': 'desire',
    '9': 'disappointment',
    '10': 'disapproval',
    '11': 'disgust',
    '12': 'embarrassment',
    '13': 'excitement',
    '14': 'fear',
    '15': 'gratitude',
    '16': 'grief',
    '17': 'joy',
    '18': 'love',
    '19': 'nervousness',
    '20': 'optimism',
    '21': 'pride',
    '22': 'realization',
    '23': 'relief',
    '24': 'remorse',
    '25': 'sadness',
    '26': 'surprise',
    '27': 'neutral'
}


# Crear una nueva columna que contenga la lista de etiquetas correspondiente a cada número
df_train['etiquetas'] = df_train['emotion'].apply(lambda x: [etiquetas[num] for num in x.split(',')])

In [17]:
df_train.head()

,text,emotion,id,etiquetas
0,My favourite food is anything I didn't have to...,27,eebbqej,[neutral]
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[anger]
3,To make her feel threatened,14,ed7ypvh,[fear]
4,Dirty Southern Wankers,3,ed0bdzj,[annoyance]


In [81]:
df_train[df_train['emotion']=='4,20']

,text,emotion,id,etiquetas,num_etiquetas
1284,"if we hurry, we can still catch the heroin craze!","4,20",eetl3jg,"[approval, optimism]",2
1344,"honestly, it’s probably for the best if he did...","4,20",edmofqz,"[approval, optimism]",2
1420,he is and will be irrelevant. no worries.,"4,20",ef8kuhi,"[approval, optimism]",2
2208,i have faith that you do it *correctly*,"4,20",ee3coph,"[approval, optimism]",2
2281,"if there's evidence besides say-so, chances ar...","4,20",ef8j6vj,"[approval, optimism]",2
...,...,...,...,...,...
40938,no problem mate. if you're looking for activit...,"4,20",eerb17d,"[approval, optimism]",2
41129,we made a big raft of inflatables on little ba...,"4,20",ef5r34b,"[approval, optimism]",2
41257,so in that case i'd be able to look at what i'...,"4,20",ednpd3m,"[approval, optimism]",2
41403,sure thing my dude. you keep on doing you. i w...,"4,20",efh5074,"[approval, optimism]",2


In [21]:
# se representa en texto en minuscula 
df_train['text'] = df_train['text'].str.lower()
df_train.head()

,text,emotion,id,etiquetas
0,my favourite food is anything i didn't have to...,27,eebbqej,[neutral]
1,"now if he does off himself, everyone will thin...",27,ed00q6i,[neutral]
2,why the fuck is bayless isoing,2,eezlygj,[anger]
3,to make her feel threatened,14,ed7ypvh,[fear]
4,dirty southern wankers,3,ed0bdzj,[annoyance]


In [85]:
df_train.iloc[2208]

text             i have faith that you do it *correctly*
emotion                                             4,20
id                                               ee3coph
etiquetas                           [approval, optimism]
num_etiquetas                                          2
Name: 2208, dtype: object

In [86]:
df_train['text'] = df_train['text'].str.replace('*', '')
df_train.iloc[2208]

<ipython-input-86-b3197ed3acbb>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train['text'] = df_train['text'].str.replace('*', '')


text             i have faith that you do it correctly
emotion                                           4,20
id                                             ee3coph
etiquetas                         [approval, optimism]
num_etiquetas                                        2
Name: 2208, dtype: object

In [54]:
# Crear una nueva columna con el número de etiquetas para cada texto
df_train['num_etiquetas'] = df_train['etiquetas'].apply(lambda x: len(x))
df_train.head()

,text,emotion,id,etiquetas,num_etiquetas
0,my favourite food is anything i didn't have to...,27,eebbqej,[neutral],1
1,"now if he does off himself, everyone will thin...",27,ed00q6i,[neutral],1
2,why the fuck is bayless isoing,2,eezlygj,[anger],1
3,to make her feel threatened,14,ed7ypvh,[fear],1
4,dirty southern wankers,3,ed0bdzj,[annoyance],1


In [55]:
df_train['num_etiquetas'].describe()

count    43410.000000
mean         1.177217
std          0.417699
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          5.000000
Name: num_etiquetas, dtype: float64

In [56]:
df_train.groupby(['num_etiquetas'])['text'].count()

num_etiquetas
1    36308
2     6541
3      532
4       28
5        1
Name: text, dtype: int64

Haya presencia de una observación que tiene asociada 5 sentimientos de un total de 28 disponibles. Esta observacion se presenata a continuación:

In [58]:
df_train[df_train['num_etiquetas'] == 5]

,text,emotion,id,etiquetas,num_etiquetas
7873,yeah i probably would've started crying on the...,"9,12,14,19,25",ee6lqiq,"[disappointment, embarrassment, fear, nervousn...",5


In [59]:
df_train[df_train['num_etiquetas'] == 5]['text'].to_list()

['yeah i probably would\'ve started crying on the spot. loud, sudden and especially shrill noises are extremely *""cringey""* and uncomfortable and stressful']

A continuación, se procede a crear la columna de etiquetas, respresentadas con 0 y 1, de acuerdo si hay presencia o no de un sentimiento.

In [87]:
from sklearn.preprocessing import MultiLabelBinarizer

# Creamos una instancia de MultiLabelBinarizer y usamos fit_transform para convertir las etiquetas en un formato binario
mlb = MultiLabelBinarizer()
etiquetas_binarias = mlb.fit_transform(df_train['etiquetas'])

# Creamos un DataFrame a partir de las etiquetas binarias y les ponemos nombres a las columnas
df_etiquetas_binarias = pd.DataFrame(etiquetas_binarias, columns=mlb.classes_)

# Concatenamos el DataFrame de etiquetas binarias con el DataFrame original
df = pd.concat([df_train, df_etiquetas_binarias], axis=1)
df.head()

,text,emotion,id,etiquetas,num_etiquetas,admiration,amusement,anger,annoyance,approval,...,love,nervousness,neutral,optimism,pride,realization,relief,remorse,sadness,surprise
0,my favourite food is anything i didn't have to...,27,eebbqej,[neutral],1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,"now if he does off himself, everyone will thin...",27,ed00q6i,[neutral],1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,why the fuck is bayless isoing,2,eezlygj,[anger],1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,to make her feel threatened,14,ed7ypvh,[fear],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dirty southern wankers,3,ed0bdzj,[annoyance],1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
labels = mlb.classes_
labels

array(['admiration', 'amusement', 'anger', 'annoyance', 'approval',
       'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
       'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral',
       'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness',
       'surprise'], dtype=object)

In [88]:
df[df['emotion']=='4,20']

,text,emotion,id,etiquetas,num_etiquetas,admiration,amusement,anger,annoyance,approval,...,love,nervousness,neutral,optimism,pride,realization,relief,remorse,sadness,surprise
1284,"if we hurry, we can still catch the heroin craze!","4,20",eetl3jg,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1344,"honestly, it’s probably for the best if he did...","4,20",edmofqz,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1420,he is and will be irrelevant. no worries.,"4,20",ef8kuhi,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2208,i have faith that you do it correctly,"4,20",ee3coph,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2281,"if there's evidence besides say-so, chances ar...","4,20",ef8j6vj,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40938,no problem mate. if you're looking for activit...,"4,20",eerb17d,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
41129,we made a big raft of inflatables on little ba...,"4,20",ef5r34b,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
41257,so in that case i'd be able to look at what i'...,"4,20",ednpd3m,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
41403,sure thing my dude. you keep on doing you. i w...,"4,20",efh5074,"[approval, optimism]",2,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


La presencia total de cada uno de los sentimientos, en el total de observaciones, se distribuye como:

In [61]:
df.iloc[:,5:].sum().sort_values(ascending = False)

neutral           14219
admiration         4130
approval           2939
gratitude          2662
annoyance          2470
amusement          2328
curiosity          2191
love               2086
disapproval        2022
optimism           1581
anger              1567
joy                1452
confusion          1368
sadness            1326
disappointment     1269
realization        1110
caring             1087
surprise           1060
excitement          853
disgust             793
desire              641
fear                596
remorse             545
embarrassment       303
nervousness         164
relief              153
pride               111
grief                77
dtype: int64

Que porcentualmente corresponde a:

In [62]:
df.iloc[:,5:].sum().sort_values(ascending = False) / df.iloc[:,5:].sum().sum() * 100

neutral           27.824198
admiration         8.081717
approval           5.751130
gratitude          5.209088
annoyance          4.833376
amusement          4.555506
curiosity          4.287420
love               4.081952
disapproval        3.956715
optimism           3.093752
anger              3.066356
joy                2.841320
confusion          2.676947
sadness            2.594760
disappointment     2.483220
realization        2.172084
caring             2.127077
surprise           2.074242
excitement         1.669178
disgust            1.551768
desire             1.254329
fear               1.166272
remorse            1.066474
embarrassment      0.592920
nervousness        0.320920
relief             0.299395
pride              0.217208
grief              0.150676
dtype: float64

Hay presencia de sentimientos con muy bajo porcentaje, incluso menor a 1%. Mientras que el sentimiento que mas presencia tiene es `neutral` 

## Tratamiento del texto

In [65]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [90]:
%%time
def clean_text(txt):
    """""
    Limpieza del texto ingresado, considerando los siguientes pasos
    1- Sustituir las contracciones
    2- Eliminar puntuación
    3- Dividir en palabras
    4- Eliminar palabras vacías (stopwords)
    5- Eliminar los signos de puntuación sobrantes
    """""
    contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                        "'cause": "because", "could've": "could have", 
                        "couldn't": "could not", "didn't": "did not",  
                        "doesn't": "does not", "don't": "do not", 
                        "hadn't": "had not", "hasn't": "has not", 
                        "haven't": "have not", "he'd": "he would",
                        "he'll": "he will", "he's": "he is", "how'd": "how did", 
                        "how'd'y": "how do you", "how'll": "how will", 
                        "how's": "how is",  "I'd": "I would", 
                        "I'd've": "I would have", "I'll": "I will", 
                        "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                        "i'd": "i would", "i'd've": "i would have", 
                        "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                        "i've": "i have", "isn't": "is not", "it'd": "it would", 
                        "it'd've": "it would have", "it'll": "it will", 
                        "it'll've": "it will have","it's": "it is", 
                        "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                        "might've": "might have","mightn't": "might not",
                        "mightn't've": "might not have", "must've": "must have", 
                        "mustn't": "must not", "mustn't've": "must not have", 
                        "needn't": "need not", "needn't've": "need not have",
                        "o'clock": "of the clock", "oughtn't": "ought not", 
                        "oughtn't've": "ought not have", "shan't": "shall not", 
                        "sha'n't": "shall not", "shan't've": "shall not have", 
                        "she'd": "she would", "she'd've": "she would have", 
                        "she'll": "she will", "she'll've": "she will have", 
                        "she's": "she is", "should've": "should have", 
                        "shouldn't": "should not", "shouldn't've": "should not have",
                        "so've": "so have","so's": "so as", "this's": "this is",
                        "that'd": "that would", "that'd've": "that would have", 
                        "that's": "that is", "there'd": "there would", 
                        "there'd've": "there would have", "there's": "there is", 
                        "here's": "here is","they'd": "they would", 
                        "they'd've": "they would have", "they'll": "they will",
                        "they'll've": "they will have", "they're": "they are",
                        "they've": "they have", "to've": "to have", 
                        "wasn't": "was not", "we'd": "we would", 
                        "we'd've": "we would have", "we'll": "we will",
                        "we'll've": "we will have", "we're": "we are", 
                        "we've": "we have", "weren't": "were not", 
                        "what'll": "what will", "what'll've": "what will have", 
                        "what're": "what are",  "what's": "what is", 
                        "what've": "what have", "when's": "when is", 
                        "when've": "when have", "where'd": "where did", 
                        "where's": "where is", "where've": "where have",  
                        "who'll": "who will", "who'll've": "who will have", 
                         "who's": "who is", "who've": "who have", 
                        "why's": "why is", "why've": "why have", 
                        "will've": "will have", "won't": "will not", 
                        "won't've": "will not have", "would've": "would have", 
                        "wouldn't": "would not", "wouldn't've": "would not have", 
                        "y'all": "you all", "y'all'd": "you all would",
                        "y'all'd've": "you all would have","y'all're": "you all are",
                        "y'all've": "you all have","you'd": "you would", 
                        "you'd've": "you would have", "you'll": "you will", 
                        "you'll've": "you will have", "you're": "you are", 
                        "you've": "you have"}

    def _get_contractions(contraction_dict):
        contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
        return contraction_dict, contraction_re

    def replace_contractions(text):
        contractions, contractions_re = _get_contractions(contraction_dict)
        def replace(match):
            return contractions[match.group(0)]
        return contractions_re.sub(replace, text)

    # Sustituir las contracciones
    txt = replace_contractions(txt)
    
    # eliminar puntuacion
    txt  = "".join([char for char in txt if char not in string.punctuation])
    txt = re.sub('[0-9]+', '', txt)
    
    # Dividir en palabras
    words = word_tokenize(txt)
    
    # Eliminar palabras vacías (stopwords)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    
    # Eliminar los signos de puntuación sobrantes
    words = [word for word in words if word.isalpha()]
    
    cleaned_text = ' '.join(words)
    return cleaned_text

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 21.2 µs


In [91]:
df['textos_limpios'] = df['text'].apply(lambda txt: clean_text(txt))
df['textos_limpios']

0                             favourite food anything cook
1        everyone think hes laugh screwing people inste...
2                                      fuck bayless isoing
3                                     make feel threatened
4                                   dirty southern wankers
                               ...                        
43405    added mate well got bow love hunting aspect ga...
43406              always thought funny reference anything
43407    talking anything bad happened name fault good ...
43408                            like baptism sexy results
43409                                           enjoy ride
Name: textos_limpios, Length: 43410, dtype: object

In [68]:
X = df['textos_limpios']
X

0                             favourite food anything cook
1        everyone think hes laugh screwing people inste...
2                                      fuck bayless isoing
3                                     make feel threatened
4                                   dirty southern wankers
                               ...                        
43405    added mate well got bow love hunting aspect ga...
43406              always thought funny reference anything
43407    talking anything bad happened name fault good ...
43408                            like baptism sexy results
43409                                           enjoy ride
Name: textos_limpios, Length: 43410, dtype: object

In [103]:
y = df.iloc[:,5:33]
y

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,neutral,optimism,pride,realization,relief,remorse,sadness,surprise
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43405,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
43406,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43407,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Propuesta de Indicador o KPI

Se suguiere como indicador, definir una escala de ponderación para cada sentimiento basada en su impacto en la satisfacción del cliente. Cada uno de los sentimientos tendrá una puntuación y grupos de sentimientos tendran una ponderación. Por ejemplo, "approval", "love" y "gratitude" debe tener un peso mayor "neutral". 

Para aquellas observaciones donde los comentarios tiene uno o más sentimientos, se debe calcular un puntaje ponderado sumando los puntajes de cada sentimiento multiplicados por su peso relativo. 

Finalmente, el nivel de satisfacción global se puede calcular como el promedio de los puntajes ponderados de todos los comentarios.



---



---



---



Se separa en conjunto de entrenamiento y de prueba

In [104]:
# train test split for protect overfitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)
print('X_train: ', X_train.shape )
print('X_test: ' , X_test.shape )
print('y_train: ', y_train.shape)
print('y_test: ' , y_test.shape)


X_train:  (39069,)
X_test:  (4341,)
y_train:  (39069, 28)
y_test:  (4341, 28)


# Modelo

Modelo 1 Random Forest

In [106]:
# pipeline
clf1 = Pipeline([("vectorizer", TfidfVectorizer(max_features = 25000)),
                ('classifier', OneVsRestClassifier(RandomForestClassifier(n_estimators= 100))),
               ])
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
y_pred_train = clf1.predict(X_train)


In [119]:
print('F1_score'  ,f1_score(y_test, y_pred1,average='samples'), 'percent')
print("Hamming loss:",hamming_loss(y_test,y_pred1)) 
print('roc_auc_score'  ,roc_auc_score(y_test, y_pred1)) 
print('accuracy_score'  ,accuracy_score(y_test, y_pred1) )
print('average_precision_score'  ,average_precision_score(y_test, y_pred1) )

F1_score 0.3550334024418336 percent
Hamming loss: 0.03605160101359134
roc_auc_score 0.6013047202894105
accuracy_score 0.30568993319511634
average_precision_score 0.17691062053366072


In [114]:
print('classification_report'  ,classification_report(y_test, y_pred1) )

classification_report               precision    recall  f1-score   support

           0       0.67      0.43      0.52       422
           1       0.75      0.74      0.75       234
           2       0.48      0.18      0.26       164
           3       0.52      0.06      0.10       288
           4       0.47      0.11      0.18       275
           5       0.67      0.09      0.16       113
           6       0.67      0.08      0.14       154
           7       0.44      0.07      0.12       229
           8       0.47      0.15      0.23        47
           9       0.62      0.06      0.10       141
          10       0.21      0.02      0.04       198
          11       0.82      0.21      0.33        87
          12       1.00      0.03      0.06        31
          13       0.41      0.14      0.21        86
          14       0.73      0.28      0.40        79
          15       0.94      0.82      0.88       267
          16       0.00      0.00      0.00         7
     

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
# pipeline
clf2 = Pipeline([("vectorizer",
                 TfidfVectorizer(max_features = 25000)), 
                ("classifier", OneVsRestClassifier(xgb.XGBClassifier()))])
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)


Evaluación del modelo

In [128]:
print('F1_score'  ,f1_score(y_test, y_pred1,average='samples'), 'percent')
print('f1_score', f1_score(y_test, y_pred1, average='macro'))
print('hamming_los'  ,hamming_loss(y_test, y_pred2))
print('roc_auc_score'  ,roc_auc_score(y_test, y_pred2)) 
print('accuracy_score'  ,accuracy_score(y_test, y_pred2) )
print('average_precision_score'  ,average_precision_score(y_test, y_pred2) )

F1_score 0.3550334024418336 percent
f1_score 0.27108151580143375
hamming_los 0.036388916312896966
roc_auc_score 0.6170992074757873
accuracy_score 0.2257544344621055
average_precision_score 0.19329302221296482


In [125]:
print('classification_report'  ,classification_report(y_test, y_pred2) )

classification_report               precision    recall  f1-score   support

           0       0.70      0.42      0.52       422
           1       0.73      0.75      0.74       234
           2       0.52      0.20      0.28       164
           3       0.57      0.07      0.13       288
           4       0.65      0.08      0.14       275
           5       0.55      0.11      0.18       113
           6       0.87      0.08      0.15       154
           7       0.82      0.06      0.11       229
           8       0.48      0.28      0.35        47
           9       0.75      0.06      0.12       141
          10       0.40      0.02      0.04       198
          11       0.67      0.30      0.41        87
          12       0.33      0.10      0.15        31
          13       0.43      0.15      0.22        86
          14       0.71      0.32      0.44        79
          15       0.93      0.82      0.87       267
          16       0.00      0.00      0.00         7
     

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




---



---



---



## Prediciones Base test

In [129]:
df_test.head()

,text,id
0,I’m really sorry about your situation :( Altho...,eecwqtt
1,It's wonderful because it's awful. At not with.,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",een27c3
3,"I didn't know that, thank you for teaching me ...",eelgwd1
4,They got bored from haunting earth for thousan...,eem5uti


In [130]:
# se representa en texto en minuscula 
df_test['text'] = df_test['text'].str.lower()
df_test['text'] = df_test['text'].str.replace('*', '')

df_test['textos_limpios'] = df_test['text'].apply(lambda txt: clean_text(txt))
df_test['textos_limpios']

<ipython-input-130-bc35e5d3a702>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_test['text'] = df_test['text'].str.replace('*', '')


0       really sorry situation although love names sap...
1                                         wonderful awful
2         kings fan good luck guys interesting game watch
3                     know thank teaching something today
4       got bored haunting earth thousands years ultim...
                              ...                        
5422             thanks diagnosed bp hospitalization well
5423                                     well makes sense
5424                                    daddy issues name
5425     glad discovered subreddit couple months ago good
5426    watch elmo grouchland one time many kids littl...
Name: textos_limpios, Length: 5427, dtype: object

In [131]:
predicciones = clf1.predict(df_test['textos_limpios'])

In [135]:
# Convertir las predicciones a etiquetas

prediccion_sentimientos = mlb.inverse_transform(predicciones)
type(prediccion_sentimientos)

list

In [140]:
prediccion_sentimientos[342]

('admiration', 'amusement')

In [ ]:
for sent in prediccion_sentimientos:
    print(sent)

### Guardar modelos

In [126]:
#guardar modelo
import pickle
pickle.dump(clf1, open('RandomForestModelOneVsRest.pkl', 'wb'))
pickle.dump(clf2, open('XGboostModelOneVsRest.pkl', 'wb'))

## Preguntas


1. ¿De qué manera se puede complementar la solución? Pensar en propuestas para el cliente.
<br>
Realizado el modelo, se suguiere generar un reporte con el nivel de satisfacción global de los clientes. Identificar palabras frecuentes en los comantarios, relacionados con cada uno de los sentimientos. 
<br>
Seleccionar aquellos comentarios negativos y analizar cuales podrían la causa, y categirizarlas por ejemplo: en mal servicio, mal producto, entre otros. 
<br>
2. ¿Cómo se podría simplificar la tarea?
<br>
Se sugiere disminuir la cantidad de etiquetas, por ejemplo se pueden agrupar los sentimientos en categorías más amplias y generales.
<br>
3. ¿Cuáles pueden ser las limitaciones, riesgos, sesgos de los modelos al implementar este tipo de soluciones? 
<br>
Los modelos de clasificación multilabel pueden estar sesgados si los datos de entrenamiento no representan adecuadamente la diversidad de sentimientos y opiniones de los clientes. A medida que se tiene una mayor cantidad de etiquetas, el modelo puede ser menos preciso y confiable.
<br>
4. ¿Qué otras cosas hay que considerar al momento de implementar un proyecto como este?
<br>
Para el procesamiento de los textos analizar la posibilidad de realizar los embedding con otras técnicas, por ejemplo Glove. Cualquiera sea la técnica de tratamiento de texto, este se debe preprocesar para eliminar ruido, errores ortográficos, caracteres especiales y palabras irrelevantes. 
<br>
Monitorear el modelo considerando los cambios y/o modismos en en lenguaje que podrían mermar el rendimiento del modelo. Más aún, si es habitual ver presencia de caracteres, que combinados representan algun sentimiento, por ejemplo ":)"


In [ ]:
# Entrenar un modelo SVM de clasificación multietiqueta con kernel lineal
classifier = OneVsRestClassifier(SVC(kernel='linear'))
classifier.fit(X, y)

# Hacer predicciones en nuevos datos
X_new = pd.DataFrame({'caracteristica_1': [1, 2, 3], 'caracteristica_2': [4, 5, 6], 'caracteristica_3': [7, 8, 9]})
y_new = classifier.predict(X_new)

# Convertir las predicciones a etiquetas
y_new_labels = mlb.inverse_transform(y_new)

# precisión
precision = precision_score(y_true, y_pred, average='micro')

# recuperación
recall = recall_score(y_true, y_pred, average='micro')

# F1-score
f1 = f1_score(y_true, y_pred, average='micro')

# exactitud
accuracy = accuracy_score(y_true, y_pred)

# AUC-ROC
auc_roc = roc_auc_score(y_true, y_pred, average='micro')

############33
from sklearn.metrics import label_ranking_average_precision_score
import numpy as np

# supongamos que las etiquetas verdaderas son y_true y las etiquetas predichas son y_pred
y_true = np.array([[1, 0, 1], [0, 1, 0], [1, 1, 1]])
y_pred = np.array([[1, 0, 0], [0, 1, 1], [1, 0, 1]])

lr_avg_precision = label_ranking_average_precision_score(y_true, y_pred)

print("Label Ranking Average Precision Score: ", lr_avg_precision)
###############

from sklearn.metrics import label_ranking_loss
import numpy as np

# supongamos que las etiquetas verdaderas son y_true y las etiquetas predichas son y_pred
y_true = np.array([[1, 0, 1], [0, 1, 0], [1, 1, 1]])
y_pred = np.array([[1, 0, 0], [0, 1, 1], [1, 0, 1]])

lr_loss = label_ranking_loss(y_true, y_pred)

print("Label Ranking Loss: ", lr_loss)